<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Guide/Customization/Tensors/Notebook3_Tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the dependencies

In [2]:
%%time
import os
import cProfile
import tempfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
keras = tf.keras

CPU times: user 299 ms, sys: 27.9 ms, total: 327 ms
Wall time: 458 ms


In [3]:
%load_ext tensorboard
try:
    %tensorflow_version 2.x
except:
    print("Tensorflow not found.")
    
print(tf.__version__)
tf.enable_eager_execution()
print(tf.executing_eagerly())
print(tfds.disable_progress_bar())
if tf.config.experimental.list_physical_devices("GPU"):
    print(tf.test.is_gpu_available())

TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.0-rc3
True
None
True


## TensorFlow tensors

    TensorFlow, as the name indicates, is a framework to define and run computations involving tensors. A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

    When writing a TensorFlow program, the main object you manipulate and pass around is the tf.Tensor. A tf.Tensor object represents a partially defined computation that will eventually produce a value. TensorFlow programs work by first building a graph of tf.Tensor objects, detailing how each tensor is computed based on the other available tensors and then by running parts of this graph to achieve the desired results.

   `A tf.Tensor has the following properties:`

* **a data type (float32, int32, or string, for example)**
* **a shape**


    Each element in the Tensor has the same data type, and the data type is always known. The shape (that is, the number of dimensions it has and the size of each dimension) might be only partially known. Most operations produce tensors of fully-known shapes if the shapes of their inputs are also fully known, but in some cases it's only possible to find the shape of a tensor at graph execution time.

    Some types of tensors are special, and these will be covered in other units of the TensorFlow guide. The main ones are:

* tf.Variable
* tf.constant
* tf.placeholder
* tf.SparseTensor


    With the exception of tf.Variable, the value of a tensor is immutable, which means that in the context of a single execution tensors only have a single value. However, evaluating the same tensor twice can return different values; for example that tensor can be the result of reading data from disk, or generating a random number.

### Rank

The rank of a tf.Tensor object is its number of dimensions. Synonyms for rank include order or degree or n-dimension. Note that rank in TensorFlow is not the same as matrix rank in mathematics.

#### Rank 0

In [0]:
mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

#### Rank 1 

In [0]:
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers = tf.Variable([3.14, 2.71], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3-4.85j, 7.5-6.23j], tf.complex64)

#### Higher Ranks

In [0]:
mymat = tf.Variable([[7],[11]],tf.int16)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([ [4, 9], [16, 25] ], tf.int32)
rank_of_squares = tf.rank(squarish_squares)
mymatC = tf.Variable([[7],[11]], tf.int32)

In [0]:
my_image = tf.zeros([10,229,229,3])

### Getting a tf.Tensor object's rank

In [0]:
r = tf.rank(my_image)

In [10]:
r

<tf.Tensor: id=99, shape=(), dtype=int32, numpy=4>

### Referring to tf.Tensor slices

In [0]:
my_scalar = cool_numbers[0]

In [12]:
print(my_scalar)

tf.Tensor(3.14, shape=(), dtype=float32)


In [0]:
my_scalar = cool_numbers[0:]

In [15]:
my_scalar

<tf.Tensor: id=109, shape=(2,), dtype=float32, numpy=array([3.14, 2.71], dtype=float32)>

### Shape

#### Changing the shape of a tf.Tensor

In [0]:
rank_three_tensor = tf.ones([3, 4, 5])
matrix = tf.reshape(rank_three_tensor, [6, 10])  
matrixB = tf.reshape(matrix, [3, -1])  
matrixAlt = tf.reshape(matrixB, [4, 3, -1])

In [0]:
yet_another = tf.reshape(matrixAlt, [13, 2, -1])  # ERROR!

#### Data Types

In [0]:
float_tensor = tf.cast(tf.constant([1, 2, 3]), dtype=tf.float32)

In [19]:
float_tensor.dtype

tf.float32

#### Evaluate tensors

In [0]:
# constant = tf.constant([1, 2, 3])
# tensor = constant * constant
# print(tensor.eval())

In [0]:
# p = tf.placeholder(tf.float32)
# t = p + 1.0
# t.eval()  # This will fail, since the placeholder did not get a value.
# t.eval(feed_dict={p:2.0})

#### Print Tensors

In [0]:
t = tf.random.uniform([10,10])

In [26]:
print(t)

tf.Tensor(
[[0.57996917 0.40112615 0.93207014 0.08208466 0.9465785  0.3280046
  0.01110685 0.37546265 0.391657   0.23375201]
 [0.63735056 0.971213   0.08812869 0.17201352 0.14214313 0.12958634
  0.19076967 0.27521837 0.7691715  0.46656835]
 [0.37726378 0.9567082  0.79219663 0.18866777 0.96874416 0.2712617
  0.28417206 0.35287035 0.13864458 0.7038201 ]
 [0.18065095 0.18216634 0.2798388  0.02747536 0.2991867  0.55034184
  0.48205173 0.25512886 0.3003174  0.6708776 ]
 [0.16843629 0.7314701  0.12264884 0.30878448 0.47293818 0.15206158
  0.35277152 0.5388968  0.84693575 0.44396973]
 [0.21508157 0.22140133 0.37425983 0.12554908 0.5542183  0.38817918
  0.43499613 0.8004998  0.9315175  0.19422185]
 [0.28105235 0.74050236 0.3940102  0.32815897 0.9276681  0.213395
  0.9862571  0.5906435  0.87877524 0.07087469]
 [0.94024837 0.7593579  0.6083857  0.4409784  0.70986223 0.62330306
  0.21467435 0.29589474 0.4531852  0.8508966 ]
 [0.51603484 0.38234055 0.7370956  0.70888543 0.46382582 0.59450793
  0.7

In [27]:
tf.Print(t, [t])

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:



<tf.Tensor: id=134, shape=(10, 10), dtype=float32, numpy=
array([[0.57996917, 0.40112615, 0.93207014, 0.08208466, 0.9465785 ,
        0.3280046 , 0.01110685, 0.37546265, 0.391657  , 0.23375201],
       [0.63735056, 0.971213  , 0.08812869, 0.17201352, 0.14214313,
        0.12958634, 0.19076967, 0.27521837, 0.7691715 , 0.46656835],
       [0.37726378, 0.9567082 , 0.79219663, 0.18866777, 0.96874416,
        0.2712617 , 0.28417206, 0.35287035, 0.13864458, 0.7038201 ],
       [0.18065095, 0.18216634, 0.2798388 , 0.02747536, 0.2991867 ,
        0.55034184, 0.48205173, 0.25512886, 0.3003174 , 0.6708776 ],
       [0.16843629, 0.7314701 , 0.12264884, 0.30878448, 0.47293818,
        0.15206158, 0.35277152, 0.5388968 , 0.84693575, 0.44396973],
       [0.21508157, 0.22140133, 0.37425983, 0.12554908, 0.5542183 ,
        0.38817918, 0.43499613, 0.8004998 , 0.9315175 , 0.19422185],
       [0.28105235, 0.74050236, 0.3940102 , 0.32815897, 0.9276681 ,
        0.213395  , 0.9862571 , 0.5906435 , 0.878775

In [29]:
t = tf.Print(t, [t])
t = t+1
tf.print(t)

[[2.57996917 2.40112615 2.93207026 ... 2.37546253 2.39165688 2.23375201]
 [2.63735056 2.97121286 2.08812857 ... 2.27521849 2.76917148 2.46656847]
 [2.37726378 2.95670819 2.79219675 ... 2.35287046 2.1386447 2.70382023]
 ...
 [2.94024849 2.75935793 2.60838556 ... 2.29589462 2.45318508 2.8508966]
 [2.51603484 2.38234043 2.73709559 ... 2.40739536 2.8508935 2.59362316]
 [2.39458036 2.32055044 2.91234636 ... 2.67290449 2.9944036 2.80406618]]
